## Imports

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')


In [3]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [4]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


## Modelling

In [6]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax')
])

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=50)


Epoch 1/50
39/39 [==============================] - 1s 1ms/step - loss: 0.8186 - accuracy: 0.4342
Epoch 2/50
39/39 [==============================] - 0s 1ms/step - loss: 0.6033 - accuracy: 0.6718
Epoch 3/50
39/39 [==============================] - 0s 1ms/step - loss: 0.4416 - accuracy: 0.9337
Epoch 4/50
39/39 [==============================] - 0s 1ms/step - loss: 0.3144 - accuracy: 0.9552
Epoch 5/50
39/39 [==============================] - 0s 2ms/step - loss: 0.2385 - accuracy: 0.9491
Epoch 6/50
39/39 [==============================] - 0s 2ms/step - loss: 0.1966 - accuracy: 0.9484
Epoch 7/50
39/39 [==============================] - 0s 1ms/step - loss: 0.1642 - accuracy: 0.9565
Epoch 8/50
39/39 [==============================] - 0s 1ms/step - loss: 0.1611 - accuracy: 0.9491
Epoch 9/50
39/39 [==============================] - 0s 1ms/step - loss: 0.1364 - accuracy: 0.9587
Epoch 10/50
39/39 [==============================] - 0s 1ms/step - loss: 0.1484 - accuracy: 0.9522
Epoch 11/50
39/39 [

In [12]:
loss, accuracy =model.evaluate(X_test, y_test)


10/10 [==============================] - 0s 2ms/step - loss: 0.1040 - accuracy: 0.9711


In [14]:
loss, accuracy

(0.10402051359415054, 0.971061110496521)

In [15]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                30        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 162
Trainable params: 162
Non-trainable params: 0
_________________________________________________________________


## Testing

In [16]:
 prediction1 = model.predict(sc.transform(np.array([[42,50000]])))[:,1]
 prediction1

array([0.7534793], dtype=float32)

In [17]:
predict2 = model.predict(sc.transform(np.array([[20,40000]])))[:,1]
predict2

array([0.00263595], dtype=float32)

## Saving

In [18]:
model.save('customer_behavior_model/1')


INFO:tensorflow:Assets written to: customer_behavior_model/1/assets


In [19]:
!ls

customer_behavior_model  sample_data


In [20]:
!ls customer_behavior_model/1

assets	saved_model.pb	variables


## Check saved model

In [23]:
from tensorflow.keras.models import load_model


In [24]:
cust_model = load_model('customer_behavior_model/1/')

In [25]:
 prediction3 = cust_model.predict(sc.transform(np.array([[42,50000]])))[:,1]
 prediction3

array([0.7534793], dtype=float32)

In [26]:
predict4 = model.predict(sc.transform(np.array([[20,40000]])))[:,1]
predict4

array([0.00263595], dtype=float32)

In [27]:
sc.transform(np.array([[20,40000]]))

array([[-1.40863043, -0.81755432]])

In [28]:
sc.transform(np.array([[42,50000]]))

array([[-0.15065689, -0.34524533]])

In [29]:
!ls

customer_behavior_model  sample_data


In [30]:
!zip -r customermodel.zip customer_behavior_model  

  adding: customer_behavior_model/ (stored 0%)
  adding: customer_behavior_model/1/ (stored 0%)
  adding: customer_behavior_model/1/assets/ (stored 0%)
  adding: customer_behavior_model/1/saved_model.pb (deflated 87%)
  adding: customer_behavior_model/1/variables/ (stored 0%)
  adding: customer_behavior_model/1/variables/variables.data-00000-of-00001 (deflated 54%)
  adding: customer_behavior_model/1/variables/variables.index (deflated 64%)


In [31]:
!ls


customer_behavior_model  customermodel.zip  sample_data


In [32]:
from google.colab import files


## Save in tf.js format

In [34]:
# !pip install tensorflowjs

In [35]:
import tensorflowjs as tfjs

In [36]:
tfjs.converters.save_keras_model(model, "/content/")

In [37]:
!ls

customer_behavior_model  group1-shard1of1.bin  sample_data
customermodel.zip	 model.json


In [38]:
!cat model.json

{"format": "layers-model", "generatedBy": "keras v2.4.0", "convertedBy": "TensorFlow.js Converter v3.6.0", "modelTopology": {"keras_version": "2.4.0", "backend": "tensorflow", "model_config": {"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 2], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "units": 10, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 10, "activation": "relu", "use_bias": true, "kernel_initiali

In [39]:
from google.colab import files


In [ ]:
files.download('model.json')

In [42]:
!ls

customer_behavior_model  group1-shard1of1.bin  sample_data
customermodel.zip	 model.json


In [ ]:
files.download('group1-shard1of1.bin')